In [ ]:
# Aquest programa agafa test-converted.csv  (test amb files expandides) i el prepara per fer les prediccions
import pandas as pd
file="test-converted"
df = pd.read_csv(f"./../data/{file}.csv",sep=",")

In [ ]:
# Similar a train-parser 
df = df[["ID","image_embedding","category","aggregated_family","length_type","family","color_rgb","archetype","moment","life_cycle_length","fabric","phase_in","num_week_iso", "num_stores","num_sizes","price","year","weekly_demand"]]
df.head(100)

In [ ]:
df["moment"].unique()


In [ ]:
# El mateix q train-parser
df["phase_in"] = pd.to_datetime(df["phase_in"], format="%d/%m/%Y")

# Semana ISO de la fecha de phase_in
df["week_iso"] = df["phase_in"].dt.isocalendar().week

# Semana ISO actual (hoy)
today = pd.Timestamp.today()
current_week = today.isocalendar().week

# Semanas que lleva a la venta
df["week_iso"] = (df["week_iso"] + df["num_week_iso"] -1) % 52

df.head(100)

In [ ]:
# Separar la columna colores en tres nuevas
df[["R", "G", "B"]] = df["color_rgb"].str.split(",", expand=True)

# Convertir a enteros (porque inicialmente son strings)
df[["R", "G", "B"]] = df[["R", "G", "B"]].astype(int)

In [ ]:
df = df.drop(columns=["color_rgb","life_cycle_length","phase_in"])
df.head()

In [ ]:
def oneHot(df, column_index):
    column_name = df.columns[column_index]
    one_hot_df = pd.get_dummies(df[column_name], prefix=column_name).astype(int)
    df = pd.concat([df, one_hot_df], axis=1)
    return df

column_index_list = [2,3,4,5,6,7,8]

# apply one-hot to each column
for i in column_index_list:
    df = oneHot(df, i)

df.head(20)
# drop original columns
#df = df.drop(df.columns[column_index_list], axis=1)

In [ ]:
# Aqui agafem les columnes de train-parser per assegurar que totes hi son
headers = ['image_embedding',
 'category',
 'aggregated_family',
 'length_type',
 'family',
 'archetype',
 'moment',
 'fabric',
 'num_week_iso',
 'num_stores',
 'num_sizes',
 'price',
 'year',
 'weekly_demand',
 'week_iso',
 'R',
 'G',
 'B',
 'category_Accesories, Swim and Intimate',
 'category_Bottoms',
 'category_Dresses, jumpsuits and Complete set',
 'category_Outerwear',
 'category_Tops',
 'aggregated_family_Coats and Parkas',
 'aggregated_family_Dresses and jumpsuits',
 'aggregated_family_Jackets and Blazers',
 'aggregated_family_Jeans',
 'aggregated_family_Shirts',
 'aggregated_family_Skirts and shorts',
 'aggregated_family_Sweaters and Cardigans',
 'aggregated_family_Swim and intimate',
 'aggregated_family_T-shirts',
 'aggregated_family_Tops',
 'aggregated_family_Trousers & leggings',
 'length_type_Ankle brace',
 'length_type_Asymmetric',
 'length_type_Capri',
 'length_type_Crop',
 'length_type_Long',
 'length_type_Maxi',
 'length_type_Medium',
 'length_type_Midi',
 'length_type_Mini/Micro',
 'length_type_Short',
 'length_type_Standard',
 'length_type_Three quarters',
 'family_Blazers',
 'family_Bodysuits',
 'family_Cardigans',
 'family_Coats',
 'family_Dresses',
 'family_Intimate',
 'family_Jackets',
 'family_Jeans',
 'family_Jumpsuit',
 'family_Leather jackets',
 'family_Leggings and joggers',
 'family_Outer Vest',
 'family_Parkas',
 'family_Poloshirts',
 'family_Puffer coats',
 'family_Shirt',
 'family_Shorts',
 'family_Skirts',
 'family_Sweater',
 'family_Sweatshirts',
 'family_Swimwear',
 'family_T-shirt',
 'family_Tops',
 'family_Trenchcoats',
 'family_Trousers',
 'family_Vest',
 'archetype_Authentic',
 'archetype_Casual Chic',
 'archetype_Cosmopolitan',
 'archetype_Expert',
 'archetype_Ideal',
 'archetype_Lady',
 'archetype_Wow',
 'moment_CASUAL EVENT',
 'moment_CASUAL WORK',
 'moment_FORMAL EVENT',
 'moment_FORMAL WORK',
 'moment_TIME OFF',
 'fabric_CIRCULAR',
 'fabric_JEANS',
 'fabric_LEATHER',
 'fabric_SYNTHETIC LEATHER',
 'fabric_TRICOT',
 'fabric_WOVEN']
for i in headers:
    if i not in df.columns:
        df[i] = 0 # Si alguna columna no hi es, la creem amb 0

df.head(20)

In [ ]:
df = df.drop(columns=["aggregated_family","family","archetype","moment","fabric","length_type","category"])
df.head()


In [ ]:
df["image_embedding"] = df["image_embedding"].str.split(",").apply(lambda x: [float(i) for i in x])

embedding_df = pd.DataFrame(df["image_embedding"].tolist(), columns=[f"f{i}" for i in range(len(df["image_embedding"][0])) ])
embedding_df.head()
df = pd.concat([df.drop(columns=["image_embedding"]), embedding_df], axis=1)

In [ ]:
df.head(20)

In [ ]:
# Una ultima comprovacio per assegurar que totes les columnes de train-parser hi son
dt = pd.read_csv("./../data/train-parsed.csv",sep=",")
c=df.columns.tolist()
for i in dt.columns.tolist():
    if i not in c:
        print(i) # Si surt algun print, malament

In [ ]:
df.to_csv("./../data/test-parsed.csv",index=False)